# Import and Merge APDS Data

This notebook imports and refines raw APDS data.  
It then merges the APDS data with merged CPR and FR24 data.

In [ ]:
import os
from datetime import timedelta
from pru.trajectory_fields import split_dual_date, iso8601_date_parser, ISO8601_DATE_FORMAT
from pru.trajectory_files import DEFAULT_STANDS_FILENAME
from pru.filesystem.data_store_operations import MERGED_DIR
from scripts.tasks import refine_apds_file
from scripts.task_wrappers import merge_apds_data

##  Input name of the APDS data file

E.g.: FAC_APDS_FLIGHT_IR691_2017-08-01_2017-09-01.csv.bz2

Note: the file must be upload/apds bucket.

In [ ]:
apds_filename = input('APDS filename: ')

## Import and Refine the APDS Data

`refine_apds_file` imports the APDS file from the bucket and converts it into
flights, positions and events in the common format.

Note: `refine_apds_file` only creates positions if a stands filename is given.

This process takes about 3 minutes.

In [ ]:
os.chdir(MERGED_DIR)
refine_apds_file(apds_filename, DEFAULT_STANDS_FILENAME)

## Merge the APDS Data with CPR and FR24 Data

The refined APDS data is merged with "overnight" merged CPR/FR24 data.

The loop below attempts to get the merged CPR/FR24 data for every day in the APDS data file.  
The CPR/FR24 data is matched and merged with the APDS data and written to the sources/merged/apds_cpr_fr24 bucket.

This process takes about 10 minutes per day.

In [ ]:
os.chdir(MERGED_DIR)
from_date, to_date = split_dual_date(apds_filename)
datet = iso8601_date_parser(from_date)
finish = iso8601_date_parser(to_date)
ok = True
while ok and (datet <= finish):
    date = datet.strftime(ISO8601_DATE_FORMAT)
    ok = merge_apds_data(from_date, to_date, str(date))
    datet += timedelta(days=1)

ok